# 📘 00 - Configuração do Ambiente e Credenciais

Este notebook tem como objetivo realizar a configuração inicial do projeto, incluindo:

- Leitura segura das credenciais da AWS
- Configuração dos acessos via Spark
- Definição dos caminhos das camadas Bronze, Silver e Gold no S3
- Validação da conexão com o bucket da AWS

---

## 🔐 Leitura das credenciais da AWS (de forma segura)

As credenciais foram armazenadas previamente no arquivo `aws_credentials.json` no diretório `dbfs:/FileStore/tables/`. O arquivo **não é compartilhado no repositório por segurança**.


In [0]:
# Ler o arquivo JSON usando Spark
credentials_df = spark.read.option("multilinea", "true").json("dbfs:/FileStore/tables/aws_credentials.json")

credentials_dict = credentials_df.collect()[0].asDict()

# Definir credenciais da AWS
aws_access_key_id = credentials_dict['aws_access_key_id']
aws_secret_access_key = credentials_dict['aws_secret_access_key']

print(f'Leitura das credenciais de forma segura pelo Spark 🔐')

Leitura das credenciais de forma segura pelo Spark 🔐


## 📦 Definição do bucket e caminhos das camadas no S3
O nome do bucket usado no projeto é:
`mvp-brasileirao-2024`

Os caminhos das camadas foram definidos usando o prefixo `s3a://`, que permite leitura e escrita via Spark:

In [0]:
# Definir o nome do bucket S3
aws_bucket_name = "mvp-brasileirao-2024"

# Definir os caminhos das camadas no S3
bronze_path = f"s3a://{aws_bucket_name}/bronze/"
silver_path = f"s3a://{aws_bucket_name}/silver/"
gold_path = f"s3a://{aws_bucket_name}/gold/"

# Exibir os caminhos para conferência
print("Caminhos definidos:")
print(f"📂 Bronze: {bronze_path} | 📂 Silver: {silver_path} | 📂 Gold: {gold_path}")


Caminhos definidos:
📂 Bronze: s3a://mvp-brasileirao-2024/bronze/ | 📂 Silver: s3a://mvp-brasileirao-2024/silver/ | 📂 Gold: s3a://mvp-brasileirao-2024/gold/


## ⚙️ Configuração das credenciais no Spark
As credenciais foram configuradas como propriedades do Spark, utilizando as opções fs.`s3a.access.key` e `fs.s3a.secret.key`:

In [0]:
# Configurar as credenciais no Spark
spark.conf.set("fs.s3a.access.key", aws_access_key_id)
spark.conf.set("fs.s3a.secret.key", aws_secret_access_key)
spark.conf.set("fs.s3a.endpoint", "s3.amazonaws.com")

print("✅ Credenciais configuradas com sucesso!")

✅ Credenciais configuradas com sucesso!


## ✅ Validação da Conexão com o S3
Por fim, foi realizado um teste de leitura nas camadas Bronze e Silver para garantir que a conexão com o bucket estava funcionando corretamente:

In [0]:
aws_bucket_name = "mvp-brasileirao-2024"

# Definir o caminho da camada Bronze
s3_path_bronze = f"s3a://{aws_bucket_name}/bronze/"
s3_path_silver = f"s3a://{aws_bucket_name}/silver/"
# Listar arquivos na camada Bronze
try:
    files_bronze = dbutils.fs.ls(s3_path_bronze)
    files_silver = dbutils.fs.ls(s3_path_silver)
    print("✅ Conexão bem-sucedida! Arquivos encontrados na camada Bronze e Silver:")
    #display(files_bronze)  # Exibir os arquivos encontrados no Databricks
    #display(files_silver)  # Exibir os arquivos encontrados no Databricks
except Exception as e:
    print("❌ Erro ao conectar ao S3:", str(e))


✅ Conexão bem-sucedida! Arquivos encontrados na camada Bronze e Silver:
